In [ ]:
import pandas as pd
import geopandas as gpd

# Paso 1: Leer archivos
# Ajusta las rutas de tus archivos
shapefile_path = "path/to/codigos_postales.shp"
csv_files = {
    2019: "path/to/usuarios_2019.csv",
    2020: "path/to/usuarios_2020.csv",
    2021: "path/to/usuarios_2021.csv",
    2022: "path/to/usuarios_2022.csv",
    2023: "path/to/usuarios_2023.csv",
    2024: "path/to/usuarios_2024.csv"
}

output_path = "path/to/conteo_usuarios_por_municipio.csv"

In [ ]:
# Leer shapefile con información de códigos postales
codigos_postales = gpd.read_file(shapefile_path)

# Seleccionar columnas necesarias del shapefile
codigos_postales = codigos_postales[['CoPo', 'Mun']]

# Paso 2: Procesar cada base de datos de usuarios
conteo_por_municipio = {}

for year, csv_path in csv_files.items():
    # Leer base de datos de usuarios
    usuarios = pd.read_csv(csv_path)
    
    # Asegurar que las columnas estén en el tipo correcto
    usuarios['CP'] = usuarios['CP'].astype(str)
    codigos_postales['CoPo'] = codigos_postales['CoPo'].astype(str)
    
    # Unir la información del municipio
    usuarios = usuarios.merge(codigos_postales, left_on='CP', right_on='CoPo', how='left')
    
    # Contar usuarios por municipio
    conteo = usuarios['Mun'].value_counts().reset_index()
    conteo.columns = ['Municipio', year]
    conteo_por_municipio[year] = conteo

# Paso 3: Combinar los conteos en un único DataFrame
conteo_final = pd.DataFrame({'Municipio': codigos_postales['Mun'].unique()})

for year, conteo in conteo_por_municipio.items():
    conteo_final = conteo_final.merge(conteo, on='Municipio', how='left')

# Reemplazar NaN con 0 para facilitar lectura
conteo_final = conteo_final.fillna(0)

# Paso 4: Exportar el resultado a CSV
conteo_final.to_csv(output_path, index=False)

print(f"El archivo de conteo se guardó en {output_path}")